In [1]:
from configparser import ConfigParser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import kalman

config = ConfigParser()
config.read("config.ini", 'UTF-8')
BTC_PATH = config.get("default", "PATH")

In [15]:
def data_prep():
    df = pd.read_pickle(BTC_PATH)

    # 対数収益率に変換
    df['log_price'] = np.log(df['cl'])

    # 欠損値処理
    df.ffill(inplace=True)
    df.bfill(inplace=True)
    return df

df_all = data_prep()

In [16]:
su = 0.001
sw = 0.001

In [23]:
batch = 20000
for i in range(len(df_all)//batch):
    df = df_all.iloc[i*batch:(i+1)*batch].copy()
    # モデル初期化
    kf = KalmanFilter(transition_matrices=[[1, 1], [0, 1]],
                    observation_matrices=[[1, 0]],
                    initial_state_mean=[0, 0],
                    initial_state_covariance=[[1, 0], [0, 1]],
                    observation_covariance=su,
                    transition_covariance=[[0, 0], [0, sw]])

    # 予測
    df = kalman.pred_confidence_interval(kf, df)

    # トレード損益シミュレーション
    df = kalman.calc_return(df)

    # 可視化
    df[["long_ret","short_ret", "both_ret"]].cumsum().plot()
    df[["both_ret"]].cumsum().plot()
    plt.show()